## Part2 Analysis: Why CNN is More Robust to Spatial Shifts

We trained two models on MNIST:

Model A: Fully Connected Neural Network (FCNN)
Model B: Convolutional Neural Network (CNN)

Results:

FCNN Accuracy (Normal): 97.47%
FCNN Accuracy (Shifted): 43.24%

CNN Accuracy (Normal): 98.95%
CNN Accuracy (Shifted): 69.58%

Observation:

The FCNN performs very well on normal MNIST images but fails drastically
when images are shifted. This happens because FCNN treats every pixel
position independently and learns position-specific patterns. When the
digit shifts, the learned pixel-to-class relationships break.

In contrast, the CNN maintains much higher accuracy even after shifting.
This is due to properties of the convolution operation:

1. Weight Sharing:
   The same filter is applied across the entire image, allowing the model
   to detect patterns regardless of location.

2. Local Receptive Fields:
   CNNs learn local spatial features such as edges and textures, which
   remain consistent even when the object moves.

3. Translation Invariance:
   Pooling and convolution make CNNs robust to spatial transformations,
   enabling recognition of the same digit even when its position changes.

Conclusion:

This experiment demonstrates why CNNs replaced FCNNs for vision tasks.
CNNs understand spatial structure, whereas FCNNs memorize pixel positions.


In [1]:
import torch
import zipfile
import os
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader


In [2]:
zip_path = os.path.expanduser("~/Downloads/MNIST.zip")

extract_path = os.path.expanduser("~/Downloads/MNIST_extracted")

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Unzipped successfully to:", extract_path)
print("Files inside folder:")
print(os.listdir(extract_path))


Unzipped successfully to: /Users/a7206035376/Downloads/MNIST_extracted
Files inside folder:
['MNIST']


In [3]:
train_path = os.path.expanduser("~/Downloads/MNIST_extracted/MNIST/mnist_train.csv")
test_path  = os.path.expanduser("~/Downloads/MNIST_extracted/MNIST/mnist_test.csv")

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

print(train_df.shape)
print(test_df.shape)


(60000, 785)
(10000, 785)


### Convert CSV to PyTorch Dataset

In [4]:
class MNISTDataset(Dataset):
    def __init__(self, dataframe, shift=False):
        self.data = dataframe
        self.shift = shift

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        row = self.data.iloc[idx].values

        label = row[0]
        image = row[1:].reshape(28, 28).astype(np.float32) / 255.0

        # shift right by 4 pixels
        if self.shift:
            image = np.pad(image, ((0,0),(4,0)))[:, :28]

        image = torch.tensor(image).unsqueeze(0)  # 1×28×28
        label = torch.tensor(label).long()

        return image, label


In [5]:
# Create datasets
train_dataset = MNISTDataset(train_df, shift=False)
test_dataset = MNISTDataset(test_df, shift=False)
shifted_test_dataset = MNISTDataset(test_df, shift=True)

#DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
shifted_test_loader = DataLoader(shifted_test_dataset, batch_size=64, shuffle=False)

#Sanity Check
images, labels = next(iter(train_loader))

print(images.shape)
print(labels.shape)


torch.Size([64, 1, 28, 28])
torch.Size([64])


### Model A: FCNN

In [6]:
class FCNN(nn.Module):
    def __init__(self):
        super(FCNN, self).__init__()

        self.fc1 = nn.Linear(28*28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):

        x = x.view(x.size(0), -1)

        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)

        return x

#Training setup
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)

fcnn_model = FCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(fcnn_model.parameters(), lr=0.001)

#FCNN Training Loop
epochs = 5

for epoch in range(epochs):

    fcnn_model.train()
    running_loss = 0

    for images, labels in train_loader:

        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = fcnn_model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

#Evaluate FCNN
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total

fcnn_normal_acc = evaluate(fcnn_model, test_loader)
print("FCNN Accuracy (Normal Test):", fcnn_normal_acc)

fcnn_shift_acc = evaluate(fcnn_model, shifted_test_loader)
print("FCNN Accuracy (Shifted Test):", fcnn_shift_acc)


Using device: mps
Epoch 1, Loss: 0.2812
Epoch 2, Loss: 0.1085
Epoch 3, Loss: 0.0723
Epoch 4, Loss: 0.0535
Epoch 5, Loss: 0.0407
FCNN Accuracy (Normal Test): 97.47
FCNN Accuracy (Shifted Test): 43.24


### Model B: CNN

In [7]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):

        x = torch.relu(self.conv1(x))
        x = self.pool(x)

        x = torch.relu(self.conv2(x))
        x = self.pool(x)

        x = x.view(x.size(0), -1)

        x = torch.relu(self.fc1(x))
        x = self.fc2(x)

        return x

#Training setup    
cnn_model = SimpleCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

#Train CNN
epochs = 5

for epoch in range(epochs):

    cnn_model.train()
    running_loss = 0

    for images, labels in train_loader:

        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = cnn_model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

#Evaluate CNN (Normal Test)
cnn_normal_acc = evaluate(cnn_model, test_loader)
print("CNN Accuracy (Normal Test):", cnn_normal_acc)

#Evaluate CNN (Shifted Test)
cnn_shift_acc = evaluate(cnn_model, shifted_test_loader)
print("CNN Accuracy (Shifted Test):", cnn_shift_acc)


Epoch 1, Loss: 0.1725
Epoch 2, Loss: 0.0499
Epoch 3, Loss: 0.0340
Epoch 4, Loss: 0.0240
Epoch 5, Loss: 0.0212
CNN Accuracy (Normal Test): 98.95
CNN Accuracy (Shifted Test): 69.58
